<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 5: Interpolación de funciones

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np
from tqdm import tqdm

from scipy.linalg import orth

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func, matriz_inversa
from anmi.T5 import polinomio_lagrange, polinomio_newton, error_lagrange, error_maximo_estocastico, roots_chebyshev
from anmi.T5 import aitken_neville, interpolacion_hermite, polinomio_generico, esplines

In [ ]:
x, y, z, a, lambda_ = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda')

Dado un conjunto de puntos (nodos) $x_0, x_1, \cdots, x_m$ podemos crear un polinomio $p$ que aproxime la función $f$ y que cumpla que $p(x_i) = f(x_i)$ para todo nodo. Existen varias situaciones de interpolación:
* El número de nodos menos 1, $m-1$ es MAYOR que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es IGUAL que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es MENOR que el máximo grado de polinomios, $n$.

En este capítulo nos centraremos cuando $m - 1 = n$.

## Interpolación de Lagrange

En esta interpolación empleamos un conjunto de funciones $L = \{l_0, l_1, \cdots, l_n\}$ correspondiente a los $n+1$ nodos $\{x_i:i = 0, 1, \cdots, n\}$. Cada $l_i$ se define como

$$l_i(x) = \prod_{k = 0 \neq i}\frac{x - x_k}{x_i - x_k}$$

Y así el polinomio resultante es

$$p(x) = \sum_{i=0}^n l_i(x)f(x_i)$$

Donde $f(x_i)$ es el valor de la función $f$ en $x_i$.

In [ ]:
help(polinomio_lagrange)

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, lista_L = polinomio_lagrange(x_vals, y_vals)

In [ ]:
expand(p)

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = [expand(p).subs(x, i) for i in x_range]


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

Ahora ploteamos cada una de las funciones

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_pol = [expand(p).subs(x, i) for i in x_range]
y_real = [f.subs(x, i) for i in x_range]


for p_i, pl in enumerate(lista_L):
    y_l = [expand(pl).subs(x, i) * y_vals[p_i] for i in x_range]
    plt.plot(x_range, y_l, label=f'$l_{p_i}$')
    plt.scatter(x_vals[p_i], y_vals[p_i])

plt.plot(x_range, y_real, label=f'{f}')
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend(bbox_to_anchor=(1.05, 0.5))

## Método de Newton
El método de Lagrange, aunque da un polinomio único, tiene una serie de problemas. En primer lugar, aunque computacionalmente no parezca complejo, el cálculo de coeficientes es tedioso y poco práctico para ciertas funciones. Además, si queremos añadir un nuevo nodo, tenemos que recalcular todas las funciones de Lagrange para el polinomio nuevo, lo cual lo vuelve bastante impráctico. 

Con el método de Newton solventamos las problemáticas estableciendo un método iterativo que incorpora un nodo por vez.

En el método se aplican los siguientes cálculos:

$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{\frac{f(x_1) - f(x_0)}{x_1 - x_0}} & \\
x_1   & f(x_1) & & \color{blue}{\frac{\frac{f(x_2) - f(x_1)}{x_2 - x_1} - \frac{f(x_1) - f(x_0)}{x_1 - x_0}}{x_2 - x_0}}\\
    & & \frac{f(x_2) - f(x_1)}{x_2 - x_1}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \frac{f(x_n) - f(x_{n-1})}{x_n - x_{n-1}} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$

O de manera compacta:
$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{f[x_0, x_1]} & \\
x_1   & f(x_1) & & \color{blue}{f[x_0, x_1, x_2]}\\
    & & \color{blue}{f[x_1, x_2]}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \color{blue}{f[x_{n-1}, x_n]} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$ 
Con
$$f[x_a, x_{a+1}, \cdots, x_j] = \frac{f[x_{a+1}, x_{a + 2}, \cdots, x_j] - f[x_{a}, x_{a+1}, \cdots, x_{j-1}]}{x_j - x_a}$$
Si denominamos $a_0, a_1, \cdots, a_n$ a los monomios en azul ($a_0 = f(x_0)$, $a_1 = \frac{f(x_1) - f(x_0)}{x_1 - x_0}$, etc.) entonces el polinomio de Newton se construye como:
$$p_n(x) = a_0 + a_1(x - x_0) + a_2(x - x_0)(x - x_1) + \cdots + a_n(x - x_0)(x - x_1)\cdots(x - x_{n-1})$$


In [ ]:
help(polinomio_newton)

In [ ]:
p, mat = polinomio_newton([S(i) for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]], [S(i) for i in [0.7, 0.8, 1, 1.15, 1.25, 1.3]], x)
mat

In [ ]:
p

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, mat = polinomio_newton(x_vals, y_vals, x, evalf=5)
p

In [ ]:
mat.evalf(4)

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = [expand(p).subs(x, i) for i in x_range]


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

## Error en la interpolación de Lagrange
Dado un polinomio de orden $n$, se tiene que el error máximo viene estimado por la expresión:
$$E(x) = f(x) - p_n(x) = \frac{(x-x_0)(x-x_1)\cdots(x-x_n)}{(n+1)!}\max_{t\in[a,b]}f^{(n+1)}(t)$$

In [ ]:
help(error_lagrange)
help(error_maximo_estocastico)

#### EJERCICIO 48

In [ ]:
f = sqrt(x)
I = [0, 2]
grado = 4

x_vals = [1, 2]
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
error_lagrange(f, np.linspace(I[0], I[1], grado), I)  # Asegurate de que el máximo de la derivada para el grado no es infinito!!!!

Este ejercicio lo vamos a resolver de manera aleatoria, usando una simulación. Tenemos que simular el error dados n puntos $x_0$, $x_1$, ..., $x_n$ aleatorios.

In [ ]:
max_error = error_maximo_estocastico(f, grado=grado, N=2500)

In [ ]:
print(1/max_error)

In [ ]:
x_vals = np.sort(np.random.rand(grado) * (I[1] - I[0]) + I[0])
x_vals = [0, 0.366, 1.333, 2]
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

p, lista_L = polinomio_lagrange(x_vals, y_vals)

x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = np.array([expand(p).subs(x, i) for i in x_range])


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")
plt.plot(x_range, list(y_pol - (max_error)),  c="#bcbcbc", alpha=0.3)
plt.plot(x_range, list(y_pol + (max_error)),  c="#bcbcbc", alpha=0.3)


# plt.fill_between(list(x_range), list(y_pol - (max_error/2)), list(y_pol + (max_error/2)))

plt.legend()

## El fenómeno de Runge y la interpolación de Chebishev
Esta información está en el texto base pero muy sucinta y mal descrita. La info para esta sección la he sacado de https://brianheinold.net/notes/An_Intuitive_Guide_to_Numerical_Methods_Heinold.pdf

El fenómeno de Runge es que cuando tomamos una función la interpolación tiende a fallar en los extremos, sobre todo si los puntos de interpolación los tomamos de manera equidistante.
Obviamente, se pueden escoger los nodos $x_0, x_1, \cdots, x_n$ de muchas maneras, pero unas y otras van a ser mejores para generar interpolaciones más veraces fuera de los extremos.


In [ ]:
f = E**(sin(4*x))
I = [0, 1]
grado = 9

x_vals_equip = np.linspace(I[0], I[1], grado)
y_vals_equip = [f.evalf(subs={x: S(i)}) for i in x_vals_equip]

x_vals_extr = [0, 0.03, 0.07, 0.15, 0.5, 0.85, 0.93, 0.97, 1]
y_vals_extr = [f.evalf(subs={x: S(i)}) for i in x_vals_extr]

In [ ]:
p_equip, lista_L = polinomio_lagrange(x_vals_equip, y_vals_equip)
p_extr, lista_L = polinomio_lagrange(x_vals_extr, y_vals_extr)

In [ ]:
expand(p_equip)

In [ ]:
expand(p_extr)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
x_range_base = np.linspace(I[0] - 0.05, I[1] + 0.05, 100)
x_range_todo = np.linspace(I[0] - 1, I[1] + 0.7, 100)

for x_range_idx, x_range in enumerate([x_range_base, x_range_todo]): 
    y_real = [f.subs(x, i) for i in x_range]
    y_pol_equip = [expand(p_equip).subs(x, i) for i in x_range]
    y_pol_extr = [expand(p_extr).subs(x, i) for i in x_range]


    axs[x_range_idx].plot(x_range, y_real, label=f'{f}')
    axs[x_range_idx].scatter(x_vals_equip, y_vals_equip, c="#bcbcbc")
    axs[x_range_idx].scatter(x_vals_extr, y_vals_extr, c="#454545")

    axs[x_range_idx].plot(x_range, y_pol_equip, label='interpolacion equiprobable', c="#bcbcbc")
    axs[x_range_idx].plot(x_range, y_pol_extr, label='interpolacion extremos', c="#454545")


    axs[x_range_idx].legend()

Vemos que aunque la interpolación equidistante sea más exacta con respecto a la función real, se aleja un montón de los extremos. Sin embargo, si tomamos puntos en el extremo, la interpolación aguanta mejor cambios en el extremo.

Por el teorema del error máximo mencionado anteriormente se deriva que la major elección de valores de $x$ son las raíces de los polinomios de Chevisheb, que se corresponden a la ecuación:
$$x_k = \frac{1}{2}(a+b) + \frac{1}{2}(b - a) \cos\left( \frac{2k - 1}{2n} \pi \right)$$
Para $k = 1, \cdots, n$

In [ ]:
help(roots_chebyshev)

In [ ]:
for n in range(2, 20):
    plt.scatter(roots_chebyshev(n, I=[0, 1]), [n] * n)

Repetimos el ejemplo anterior, pero ahora tomando las raíces de Chebyshev

In [ ]:
x_vals_chev = [N(i, 6) for i in roots_chebyshev(grado, I=I)]
y_vals_chev = [f.evalf(subs={x: S(i)}) for i in x_vals_chev]

In [ ]:
p_chev, lista_L = polinomio_lagrange(x_vals_chev, y_vals_chev)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
x_range_base = np.linspace(I[0] - 0.05, I[1] + 0.05, 100)
x_range_todo = np.linspace(I[0] - 1, I[1] + 0.7, 100)

for x_range_idx, x_range in enumerate([x_range_base, x_range_todo]): 
    y_real = [f.subs(x, i) for i in x_range]
    y_pol_equip = [expand(p_equip).subs(x, i) for i in x_range]
    y_pol_chev = [expand(p_chev).subs(x, i) for i in x_range]
    y_pol_extr = [expand(p_extr).subs(x, i) for i in x_range]


    axs[x_range_idx].plot(x_range, y_real, label=f'{f}')
    axs[x_range_idx].scatter(x_vals_equip, y_vals_equip, c="#bcbcbc")
    axs[x_range_idx].scatter(x_vals_extr, y_vals_extr, c="#454545")
    axs[x_range_idx].scatter(x_vals_chev, y_vals_chev, c="#800000")


    axs[x_range_idx].plot(x_range, y_pol_equip, label='equiprobable', c="#bcbcbc")
    axs[x_range_idx].plot(x_range, y_pol_extr, label='extremos', c="#454545")
    axs[x_range_idx].plot(x_range, y_pol_chev, label='chebyshev', c="#800000")


    axs[x_range_idx].legend()

Aunque los resultados en los extremos sean peor para Chebyshev que para la interpolación de extremos, se ve que en el medio la interpolación de Chebyshev es prácticamente idéntica a la equiprobable. Esto es importante porque para esta función las diferencias en el centro para la interpolación de extremos no son muy evidentes, pero para otras funciones puede que sean peores, y en ese caso la selección de nodos por Chebyshev es ese punto intermedio.

## Algoritmos de Aitken y Neville
Estos dos algoritmos permiten reducir el coste de la computación del polinomio de interpolación. Con estos algoritmos, en lugar de calcular el polinomio de interpolación, se calcula directamente el valor de la función en un punto $x_m$ sabiendo los valores de la función en $n$ puntos diferentes.

In [ ]:
help(aitken_neville)

In [ ]:
# obtenido de https://rpubs.com/aaronsc32/nevilles-method-polynomial-interpolation
N(aitken_neville([8.1, 8.3, 8.6, 8.7], [16.9446, 17.56492, 18.50515, 18.82091], 8.4, modo='neville'))

In [ ]:
N(aitken_neville([8.1, 8.3, 8.6, 8.7], [16.94, 17.56, 18.5, 18.82], 8.4, modo='aitken'))

#### EJERCICIO 49

In [ ]:
N(aitken_neville([-2, -1, 0, 1], [-1, -0.5, 0, 0.5], 0.5, modo='neville'))

In [ ]:
N(aitken_neville([-2, -1, 0, 1], [-1, -0.5, 0, 0.5], 0.5, modo='aitken'))

## Interpolación compuesta

#### Figura 5.1

La figura muestra la interpolación de la función de Runge, $f(x) = \frac{1}{1+x^2}$, que vemos falla en los extremos bastante fuertemente.

In [ ]:
f = 1 / (1 + x ** 2)
I = [-5, 5]
x_vals = np.linspace(I[0], I[1], 11)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

p, lista_L = polinomio_lagrange(x_vals, y_vals)

x_range = np.linspace(I[0], I[1], 250)
y_real, y_pol = [f.subs(x, i) for i in x_range], [expand(p).subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

In [ ]:
x_vals = [N(i, 6) for i in roots_chebyshev(11, I=I)]
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

p, lista_L = polinomio_lagrange(x_vals, y_vals)

x_range = np.linspace(I[0], I[1], 250)
y_real, y_pol = [f.subs(x, i) for i in x_range], [expand(p).subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

## Interpolación de Hermite

La interpolación de Hermite emplea el método de Newton para construir el polinomio. En este caso se emplean las derivadas de $f$ para aportar más estabilidad y así conseguir un polinomio que sea tanto continuo él como su derivada. En este caso la base es, para $\{x_0, x_1, \cdots, x_n\}$, la siguiente: 
$\{\omega_0 = 1, \omega_1 = (x-x_0), \omega_2 = (x-x_0)^2, \omega_3 = (x-x_0)^2(x-x_1), \omega_4 = (x-x_0)^2(x-x_1)^2, \cdots, 
\omega_{2n}=(x-x_0)^2(x-x_1)^2\cdots(x-x_n), \omega_{2n+1}=(x-x_0)^2(x-x_1)^2\cdots(x-x_n)^2\}$

En este caso, la variación del método de Newton es que incluimos la información de $f$ y $f'$ en la tabla de diferencias (mostramos para $x_0, x_1, x_2$):


$$
\begin{matrix}
x_0 & f(x_0) &   &  & & &  \\
          &  & f[x_0, x_0] = f'(x_0)  &   & & &\\
x_0 & f(x_0) &                        &  f[x_0, x_0, x_1]  & & &\\
          &  & f[x_0, x_1]  &                              &    f[x_0, x_0, x_1, x_1] & & \\
x_1 & f(x_1) &                        &  f[x_0, x_1, x_1] &                                   & f[x_0, x_0, x_1, x_1, x_2] &\\
          &  & f[x_1, x_1] = f'(x_1)  &                    &    f[x_0, x_1, x_1, x_2]&                                     & f[x_0, x_0, x_1, x_1, x_2, x_2]   \\
x_1 & f(x_1) &                        &  f[x_1, x_1, x_2] &                                   & f[x_0, x_1, x_1, x_2, x_2] &\\
          &  & f[x_1, x_2]  &                              &    f[x_1, x_1, x_2, x_2]  & & \\
x_2 & f(x_2) &                        &  f[x_1, x_2, x_2]& & & \\
          &  & f[x_2, x_2] = f'(x_2)  &   & & &\\
x_2 & f(x_2) &                        &  & & & \\          
\end{matrix}
$$ 

$$p(x) = f(x_0) + f[x_0, x_0] \omega_1 + f[x_0, x_0, x_1] \omega_2 + f[x_0, x_0, x_1, x_1] \omega_3 + f[x_0, x_0, x_1, x_1, x_2] \omega_4 + 
    f[x_0, x_0, x_1, x_1, x_2, x_2] \omega_5=$$
    
$$f(x_0) + f[x_0, x_0] (x-x_0) + f[x_0, x_0, x_1] (x-x_0)^2 + f[x_0, x_0, x_1, x_1] (x-x_0)^2(x-x_1) + f[x_0, x_0, x_1, x_1, x_2] (x-x_0)^2(x-x_1)^2 + f[x_0, x_0, x_1, x_1, x_2, x_2] (x-x_0)^2(x-x_1)^2(x-x_2)$$

La ventaja de esta interpolación es que se pueden hacer las pirámides con tantas derivadas como se quiera. Puedes ver más info en https://en.wikipedia.org/wiki/Hermite_interpolation

In [ ]:
help(interpolacion_hermite)

#### EJERCICIO 50

In [ ]:
p, mat = interpolacion_hermite(x_vals=[1, 2, 5], y_vals = [-1, -3, 3], diff_vals=[-3, -1, 2], var=x)

In [ ]:
mat

In [ ]:
p

## Método de polinomio genérico
Este método está descrito en el ejercicio 56. Simplemente consiste en desarrollar un polinomio con $n$ incógnitas dando $n$ condiciones que se cumplen de la función o de sus sucesivas derivadas. Por ejemplo, para el ejecicio 56 hay 4 condiciones:
* $f(0) = 1$
* $f'(0.1) = -\frac{10}{3}$
* $f'(0.2) = -\frac{10}{3}$
* $f(0.3) = 0.001$

Por tanto, podemos construir el polinomio $p(x) = a_0 + a_1x + a_2x^2 + a_3x^3$ que satisfaga todas las soluciones, y con eso resolver el sistema.
$$D a = f$$

In [ ]:
help(polinomio_generico)

#### EJERCICIO 56

In [ ]:
p, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(0, 1), (0.3, 0.001)], [(0.1, -10/3), (0.2, -10/3)]])
p

#### EJERCICIO 50

In [ ]:
p, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(1, -1), (2, -3), (5, 3)], [(1, -3), (2, -1), (5, 2)]])
p

In [ ]:
p.subs(x, 1), p.subs(x, 2), p.subs(x, 5)

In [ ]:
p.diff(x).subs(x, 1), p.diff(x).subs(x, 2), p.diff(x).subs(x, 5)

#### EJERCICIO 51

In [ ]:
p, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(-3, 1), (3, 1)], [(1, 2), (-3, 1)]])
p

#### Conclusión

Vamos a extender los límites de este algoritmo con alguna función específica, como la de Runge, y ver si conseguimos alguna interpolación mejor, empleando valores de la función, valores de la primera derivada y valores de la segunda derivada. Para el sampleo de nodos, vamos a tomar los nodos de manera equidistante, y con los nodos del polinomio de Chebyshev.

Por tanto, vamos a tener 6 condiciones: el uso de $f$, $f'$ y $f''$, y nodos equidistantes o por Chebyshev.

In [ ]:
I = [-5, 5]
n = 11
f = 1 / (1 + x ** 2)

fdiff = f.diff(x)
fdiff2 = fdiff.diff(x)

In [ ]:
x_equi = np.linspace(I[0], I[1], n)
x_chev = [N(i, 1) for i in roots_chebyshev(n, I=I)]

In [ ]:
p_equi_f, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_equi], ])
p_equi_fdiff, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_equi], [(i, fdiff.subs(x, i)) for i in x_equi]])
p_equi_fdiff2, D_equi, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_equi], [(i, fdiff.subs(x, i)) for i in x_equi], 
                                                                      [(i, fdiff2.subs(x, i)) for i in x_equi]])

In [ ]:
p_chev_f, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_chev], ])
p_chev_fdiff, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_chev], [(i, fdiff.subs(x, i)) for i in x_chev]])
p_chev_fdiff2, D_chev, rhs, a_vals = polinomio_generico(lista_condiciones=[[(i, f.subs(x, i)) for i in x_chev], [(i, fdiff.subs(x, i)) for i in x_chev], 
                                                                      [(i, fdiff2.subs(x, i)) for i in x_chev]])

Vamos a ver cómo se comportan p, p', y p'' cuando derivamos f. Recordemos que p_equi_f debería cumplir que p = f para los puntos de interés, pero no p = f' o p = f''. Con p_equi_fdiff2, sin embargo, se tiene que cumplir que p = f, p' = f' y p'' = f''.

Empleamos los nodos equidistantes para la visualización porque para la chebyshev las matrices devuelven inversas muy instables y los polinomios no salen correctamente.

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(9, 9))

x_range = np.linspace(I[0], I[1], 250)

list_f = [f, fdiff, fdiff2]
list_p = [p_equi_f, p_equi_fdiff, p_equi_fdiff2]

list_p_label_row = ["p0", "p1", "p2"]
list_p_label_col = ["", "'", "''"]

y_list_label = ["f", "f'", "f''"]

for ax_row in range(3):
    for ax_col in range(3):
        p = list_p[ax_row]
        for _ in range(ax_col):
            p = p.diff(x)
        
        axs[ax_row][ax_col].scatter(x_equi, [list_f[ax_col].evalf(subs={x: S(i)}) for i in x_equi])
        axs[ax_row][ax_col].plot(x_range, [list_f[ax_col].evalf(subs={x: S(i)}) for i in x_range], label=y_list_label[ax_col], c='#bcbcbc')
        axs[ax_row][ax_col].plot(x_range, [p.evalf(subs={x: S(i)}) for i in x_range], 
                                 label=list_p_label_row[ax_row]+list_p_label_col[ax_col], c='#000080')
        
        min_y, max_y = float(np.min([list_f[ax_col].evalf(subs={x: S(i)}) for i in x_range])), float(np.max([list_f[ax_col].evalf(subs={x: S(i)}) for i in x_range]))
        axs[ax_row][ax_col].set_ylim(bottom=min_y - 0.3 * (max_y - min_y), top=max_y + 0.3 * (max_y - min_y))
        axs[ax_row][ax_col].legend()
plt.tight_layout()

Vemos que, a costa de hacer el polinomio más complejo y, a la vez más inestable, conseguimos que las condiciones de la primera y segunda derivadas vayan cumpliéndose. En tal caso, igual conviene tomar menos nodos para evitar tener polinomios tan complejos. 

## Método de esplines cúbicos

Los esplines son funciones de interpolación muy útiles porque dan un resultado continuo y doblemente derivable (clase $C^2$) para una serie de puntos $a = x_0 < x_1 < \cdots < x_{n-1} < x_n = b$. Esto es muy util en casos como el de antes, donde un polinomio para varios nodos puede dar un resultado inestable o difícil de computar. Los esplines son $n-1$ polinomios cúbicos que satisfacen la siguiente expresión:

$$S_i(x) = d_i + c_i(x-x_i) + b_i(x-x_i)^2 + a_i(x-x_i)^3$$

Obviamente, cada valor $d_i = f(x_i)$. En total tenemos $3(n-1)$ coeficientes que determinar, por lo que necesitamos un sistema de $3(n-1)$ ecuaciones. Para ello establecemos las siguientes condiciones:
* $S_i(x_{i+1}) = y_{i+1}$, de manera que se satisface la continuidad entre esplines.
* $S_i'(x_{i+1}) = S_{i+1}'(x_{i+1})$, de manera que se satisface que la pendiente es la misma entre dos esplines contiguos, y no hay puntos angulosos entre todos los esplines.
* $S_i''(x_{i+1}) = S_{i+1}''(x_{i+1})$, lo cual garantiza también una continuidad de la segunda derivada.

En este punto nos quedan dos sistemas de ecuaciones para llegar al deseado. Para ello podemos establecer las condiciones del primer y último esplín. En el caso más clásico, el esplín natural, $s_0''(x_0) = x_{n-1}''(x_n) = 0$.

Para determinar los valores $a_i, b_i, c_i$ vamos a resolver el siguiente sistema:
$$
\begin{bmatrix}
r_0 & 2 & 1-r_0 & \cdots & 0 & 0 & 0 \\
0 & r_1 & 2 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 2  & 1-r_{n-3} & 0 \\
0 & 0 & 0 & \cdots & r_{n-2} & 2 & 1-r_{n-2} \\
\end{bmatrix}_{n-2 \times n}
\begin{bmatrix}
z_0\\
z_1\\
\vdots\\
z_n\\
\end{bmatrix}_{n\times 1} = 
6\begin{bmatrix}
f[x_0, x_1, x_2]\\
f[x_1, x_2, x_3]\\
\vdots\\
f[x_{n-2}, x_{n-1}, x_n]\\
\end{bmatrix}_{n-2\times 1}
$$

Como el sistema de ecuaciones es indeterminado (hay 2 incógnitas más que ecuaciones), vamos imponer las condiciones de esplín natural, resultando en:

$$
\begin{bmatrix}
2 & 1-r_0 & \cdots & 0 & 0\\
r_1 & 2 & \cdots & 0 & 0\\
\vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & \cdots & 2  & 1-r_{n-3}\\
0 & 0 & \cdots & r_{n-2} & 2\\
\end{bmatrix}_{n-2 \times n-2}
\begin{bmatrix}
z_1\\
\vdots\\
z_{n-1}\\
\end{bmatrix}_{n-2\times 1} = 
6\begin{bmatrix}
f[x_0, x_1, x_2]\\
f[x_1, x_2, x_3]\\
\vdots\\
f[x_{n-2}, x_{n-1}, x_n]\\
\end{bmatrix}_{n-2\times 1}
$$

En este sistema, definimos $r_i  = \frac{h_i}{h_i + h_{i+1}}$ donde $h_i = x_{i+1} - x_i$. Los valores $a_i, b_i, c_i$ se obtienen a partir de las siguientes relaciones con $z$:
* $a_i = \frac{z_{i+1} - z_i}{6h_i}$
* $b_i = \frac{z_i}{2}$
* $c_i = \frac{y_{i+1} - y_i}{h_i} - \frac{2z_i + z_{i+1}}{6}h_i$

Y los valores $f[x_i, x_{i+1}, x_{i+2}]$ los obtenemos con el método de Newton.

In [ ]:
help(esplines)

In [ ]:
x_vals, y_vals = [0, 0.25, 0.5, 0.75, 1], [0, 0.7071, 1, 0.7071, 0]

In [ ]:
S_dict, valores, D, z, rhs = esplines(x_vals, y_vals)

In [ ]:
valores

In [ ]:
plt.scatter(x_vals, y_vals)
for i in range(len(x_vals) - 1):
    p = S_dict[f'S_{i}']
    x_range = np.linspace(x_vals[i], x_vals[i+1], 100)
    y_range = [p.subs(x, x_i) for x_i in x_range]
    
    plt.plot(x_range, y_range)

Ahora vamos a aproximar la función de runge por esplines

In [ ]:
f = 1 / (1 + x ** 2)
I = [-5, 5]
n = 9
x_vals = np.linspace(I[0], I[1], n)
y_vals = [f.subs(x, xi) for xi in x_vals]

In [ ]:
S_dict, valores, D, z, rhs = esplines(x_vals, y_vals)

In [ ]:
plt.scatter(x_vals, y_vals)
plt.plot(np.linspace(I[0], I[1], 250), [f.subs(x, xi) for xi in np.linspace(I[0], I[1], 250)])
for i in range(len(x_vals) - 1):
    p = S_dict[f'S_{i}']
    x_range = np.linspace(x_vals[i], x_vals[i+1], 100)
    y_range = [p.subs(x, x_i) for x_i in x_range]
    
    plt.plot(x_range, y_range)

Vemos que la aproximación por esplines mapea muy bien para valores de 11 en adelante.

## Ejercicios

### Ejercicio 53
Estimar el error que se comete al calcular $e^{\sqrt{x}}$ interpolando el valor de la función en dos puntos $x_0$ y $x_1$ arbitrarios en el intervalo $[1, 2]$.

Recordemos que el error viene dado por

$$E(x) = f(x) - p_n(x) = \frac{(x-x_0)(x-x_1)\cdots(x-x_n)}{(n+1)!}\max_{t\in[a,b]}f^{(n+1)}(t)$$

En este caso, para dos puntos la fórmula queda como:
$$E(x) = \left\vert\frac{(x-x_0)(x-x_1)}{2}\right\vert \left\vert \max_{t\in[1,2]}f''(t) \right\vert$$

Para ello separamos la parte del polinomio y el de la derivada, que son sus dos a maximizar.

La derivada a maximizar es:

In [ ]:
f = E ** sqrt(x)
ff = factor(f.diff(x).diff(x))
ff

In [ ]:
N(ff.subs(x, 2))

Como $x\sqrt{x} > x$ para $x > 0$, la función derivada es monótona creciente en el intervalo. Por tanto, 
$$\left\vert \max_{t\in[1,2]}f''(t) \right\vert < \left\vert f''(2)  \right\vert < 0.1506$$

In [ ]:
x_r = np.linspace(1, 2, 100)
plt.plot(x_r, [ff.subs(x, i) for i in x_r])

Por otro lado, para  $\max_{t\in[1,2]}\left\vert(t-x_0)(t-x_1)\right\vert$ tenemos que 

$$\max_{t\in[1,2]}\left\vert(t-x_0)(t-x_1)\right\vert = 
\max_{t\in[1,2]}\left\vert t^2 - 2t(x_0 + x_1) + x_0x_1\right\vert  \rightarrow_{t = \frac{x_0+x_1}{2}} \rightarrow\left(\frac{x_0+x_1}{2}-x_0\right)\left(x_1 - \frac{x_0+x_1}{2}\right) = 
\left(\frac{x_1-x_0}{2}\right)\left(\frac{x_1 - x_0}{2}\right) = \left(\frac{x_1 - x_0}{2}\right)^2 \le \frac{1}{4} $$ 

Pues la máxima diferencia sucede con $x_1 = 2, x_0 = 1$.

Por tanto, $|E(x)| \le \frac{1}{2}\frac{1}{4} 0.1506 = 0.018825 $



In [ ]:
help(polinomio_lagrange)

In [ ]:
error_lagrange(E**sqrt(x), [Symbol('x_0'), Symbol('x_1')], I=[1,2])  # Asegurate de que el máximo de la derivada para el grado no es infinito!!!!

### Ejercicio 55
Construir el polinomio de interpolación de la función de Heaviside en los nodos $[-1, -1/2, -1/3, 0, 1/3, 1/2, 1]$

In [ ]:
def heaviside(x):
    if x < 0:
        return 0
    else:
        return 1

In [ ]:
x_vals = [-S(1), -S(1)/S(2), -S(1)/S(3), 0, S(1)/S(3), S(1)/S(2), S(1)]
y_vals = [heaviside(i) for i in x_vals]
y_vals

In [ ]:
pol, mat = polinomio_newton(x_vals, y_vals)
expand(pol, frac=True)

In [ ]:
mat

### Ejercicio 56
La función $y=f(x)$ es conocida solamente en los siguientes valores:
$$
f(0)=1, \quad f^{\prime}(0.1)=-\frac{10}{3}, \quad f^{\prime}(0.2)=-\frac{10}{3}, \quad f(0.3)=0.001 .
$$
Esta función tiene un punto de inflexión en el intervalo $0<x<0.3$. Calcular una aproximación de la abscisa de dicho punto usando interpolación mediante polinomios.

In [ ]:
p, D, rhs, a_vals = polinomio_generico(lista_condiciones=[[(S(0), S(1)), (S(0.3), S(0.001))], [(S(0.1), -S(10)/S(3)), (S(0.2), -S(10)/S(3))]])
nsimplify(p, tolerance=0.001, full=True)

In [ ]:
D

In [ ]:
rhs

In [ ]:
nsimplify(a_vals, tolerance=0.001, full=True)

### Ejercicios 58
Dada la siguiente tabla de valores:
$$
\begin{matrix}
x & 0 & 1 & 2 & 3 & 4 \\
f(x) & 0 & 0 & 2 & 4 & 6
\end{matrix}
$$
calcular el esplin cúbico natural de interpolación a estos datos en el intervalo $[0,1] .$

In [ ]:
x_vals, y_vals = [S(i) for i in [0, 1, 2, 3, 4]], [S(i) for i in [0, 0, 2, 4, 6]]

In [ ]:
S_dict, valores, D, z, rhs = esplines(x_vals, y_vals)

In [ ]:
S_dict

In [ ]:
valores

In [ ]:
D

In [ ]:
z

In [ ]:
rhs